In [1]:
from keras.layers import Input, Conv2D, Flatten, Dense, Conv2DTranspose, Reshape, Lambda, Activation, BatchNormalization, LeakyReLU, Dropout, ZeroPadding2D, UpSampling2D
# from keras.layers.merge import _Merge
from keras.datasets import mnist
from keras.models import Model, Sequential
from keras import backend as K
from keras.optimizers import Adam, RMSprop
from keras.utils import plot_model
from keras.initializers import RandomNormal

import numpy as np
import json
import os
import pickle as pkl
import matplotlib.pyplot as plt

In [2]:
class GAN():
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build and compile the generator
        self.generator = self.build_generator()
        self.generator.compile(loss='binary_crossentropy', optimizer=optimizer)

        # The generator takes noise as input and generated imgs
        z = Input(shape=(100,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The valid takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator) takes
        # noise as input => generates images => determines validity
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_generator(self):

        noise_shape = (100,)

        model = Sequential()

        model.add(Dense(256, input_shape=noise_shape))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        noise = Input(shape=noise_shape)
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        img_shape = (self.img_rows, self.img_cols, self.channels)

        model = Sequential()

        model.add(Flatten(input_shape=img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        img = Input(shape=img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, save_interval=50):

        # Load the dataset
        (X_train, _), (_, _) = mnist.load_data()

        # Rescale -1 to 1
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train, axis=3)

        half_batch = int(batch_size / 2)

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random half batch of images
            idx = np.random.randint(0, X_train.shape[0], half_batch)
            imgs = X_train[idx]

            noise = np.random.normal(0, 1, (half_batch, 100))

            # Generate a half batch of new images
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)


            # ---------------------
            #  Train Generator
            # ---------------------

            noise = np.random.normal(0, 1, (batch_size, 100))

            # The generator wants the discriminator to label the generated samples
            # as valid (ones)
            valid_y = np.array([1] * batch_size)

            # Train the generator
            g_loss = self.combined.train_on_batch(noise, valid_y)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % save_interval == 0:
                self.save_imgs(epoch)

    def save_imgs(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, 100))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("gan/images/mnist_%d.png" % epoch)
        plt.close()


In [5]:
gan = GAN()
gan.train(epochs=5000, batch_size=32, save_interval=200)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_14 (Dense)             (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_10 (LeakyReLU)   (None, 512)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_11 (LeakyReLU)   (None, 256)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
________________________________________________

96 [D loss: 0.006254, acc.: 100.00%] [G loss: 4.288031]
97 [D loss: 0.011711, acc.: 100.00%] [G loss: 4.229034]
98 [D loss: 0.009825, acc.: 100.00%] [G loss: 4.203686]
99 [D loss: 0.028078, acc.: 100.00%] [G loss: 4.323985]
100 [D loss: 0.009750, acc.: 100.00%] [G loss: 4.381823]
101 [D loss: 0.013228, acc.: 100.00%] [G loss: 4.489654]
102 [D loss: 0.017655, acc.: 100.00%] [G loss: 4.517828]
103 [D loss: 0.011813, acc.: 100.00%] [G loss: 4.319728]
104 [D loss: 0.021859, acc.: 100.00%] [G loss: 4.400594]
105 [D loss: 0.025312, acc.: 100.00%] [G loss: 4.590322]
106 [D loss: 0.014651, acc.: 100.00%] [G loss: 4.606590]
107 [D loss: 0.028770, acc.: 100.00%] [G loss: 3.779186]
108 [D loss: 0.069961, acc.: 100.00%] [G loss: 4.168043]
109 [D loss: 0.018740, acc.: 100.00%] [G loss: 4.739135]
110 [D loss: 0.007373, acc.: 100.00%] [G loss: 4.749081]
111 [D loss: 0.014473, acc.: 100.00%] [G loss: 4.612603]
112 [D loss: 0.016211, acc.: 100.00%] [G loss: 4.764546]
113 [D loss: 0.018541, acc.: 100.00

244 [D loss: 0.985678, acc.: 37.50%] [G loss: 1.406699]
245 [D loss: 0.586298, acc.: 62.50%] [G loss: 1.813121]
246 [D loss: 0.689770, acc.: 59.38%] [G loss: 1.345852]
247 [D loss: 0.781508, acc.: 43.75%] [G loss: 1.609558]
248 [D loss: 1.266092, acc.: 21.88%] [G loss: 0.624302]
249 [D loss: 0.467572, acc.: 68.75%] [G loss: 1.661291]
250 [D loss: 0.661451, acc.: 53.12%] [G loss: 1.469896]
251 [D loss: 0.782645, acc.: 46.88%] [G loss: 1.164564]
252 [D loss: 0.769807, acc.: 50.00%] [G loss: 0.837999]
253 [D loss: 0.542002, acc.: 65.62%] [G loss: 1.355166]
254 [D loss: 0.830152, acc.: 40.62%] [G loss: 1.108779]
255 [D loss: 0.741938, acc.: 46.88%] [G loss: 0.927673]
256 [D loss: 0.652570, acc.: 59.38%] [G loss: 1.093816]
257 [D loss: 0.746958, acc.: 50.00%] [G loss: 0.978830]
258 [D loss: 0.744850, acc.: 43.75%] [G loss: 1.009974]
259 [D loss: 0.727317, acc.: 53.12%] [G loss: 0.914825]
260 [D loss: 0.801490, acc.: 46.88%] [G loss: 0.961529]
261 [D loss: 0.588250, acc.: 68.75%] [G loss: 1.

393 [D loss: 0.662317, acc.: 56.25%] [G loss: 0.716813]
394 [D loss: 0.668208, acc.: 43.75%] [G loss: 0.714128]
395 [D loss: 0.693159, acc.: 50.00%] [G loss: 0.672006]
396 [D loss: 0.648581, acc.: 59.38%] [G loss: 0.668749]
397 [D loss: 0.647221, acc.: 56.25%] [G loss: 0.682698]
398 [D loss: 0.644023, acc.: 53.12%] [G loss: 0.663220]
399 [D loss: 0.650402, acc.: 56.25%] [G loss: 0.665651]
400 [D loss: 0.574172, acc.: 75.00%] [G loss: 0.657506]
401 [D loss: 0.669235, acc.: 59.38%] [G loss: 0.696588]
402 [D loss: 0.589320, acc.: 78.12%] [G loss: 0.731239]
403 [D loss: 0.604625, acc.: 68.75%] [G loss: 0.721420]
404 [D loss: 0.668795, acc.: 62.50%] [G loss: 0.709680]
405 [D loss: 0.670100, acc.: 59.38%] [G loss: 0.683992]
406 [D loss: 0.641845, acc.: 56.25%] [G loss: 0.670419]
407 [D loss: 0.672106, acc.: 50.00%] [G loss: 0.665956]
408 [D loss: 0.668656, acc.: 53.12%] [G loss: 0.672243]
409 [D loss: 0.654630, acc.: 62.50%] [G loss: 0.681703]
410 [D loss: 0.661833, acc.: 56.25%] [G loss: 0.

541 [D loss: 0.661004, acc.: 62.50%] [G loss: 0.704737]
542 [D loss: 0.629547, acc.: 65.62%] [G loss: 0.677698]
543 [D loss: 0.679638, acc.: 53.12%] [G loss: 0.653304]
544 [D loss: 0.624209, acc.: 59.38%] [G loss: 0.713601]
545 [D loss: 0.627882, acc.: 56.25%] [G loss: 0.698638]
546 [D loss: 0.703218, acc.: 46.88%] [G loss: 0.730370]
547 [D loss: 0.644693, acc.: 59.38%] [G loss: 0.743071]
548 [D loss: 0.691879, acc.: 59.38%] [G loss: 0.714950]
549 [D loss: 0.673618, acc.: 65.62%] [G loss: 0.709976]
550 [D loss: 0.666544, acc.: 68.75%] [G loss: 0.698398]
551 [D loss: 0.624043, acc.: 65.62%] [G loss: 0.716629]
552 [D loss: 0.652172, acc.: 65.62%] [G loss: 0.705586]
553 [D loss: 0.650857, acc.: 65.62%] [G loss: 0.716783]
554 [D loss: 0.655405, acc.: 56.25%] [G loss: 0.703256]
555 [D loss: 0.628929, acc.: 62.50%] [G loss: 0.717968]
556 [D loss: 0.632307, acc.: 65.62%] [G loss: 0.714881]
557 [D loss: 0.635046, acc.: 62.50%] [G loss: 0.699618]
558 [D loss: 0.661374, acc.: 59.38%] [G loss: 0.

...

4313 [D loss: 0.666566, acc.: 65.62%] [G loss: 0.835719]
4314 [D loss: 0.648371, acc.: 59.38%] [G loss: 0.833838]
4315 [D loss: 0.676321, acc.: 59.38%] [G loss: 0.858352]
4316 [D loss: 0.665721, acc.: 59.38%] [G loss: 0.887659]
4317 [D loss: 0.686752, acc.: 62.50%] [G loss: 0.821596]
4318 [D loss: 0.697395, acc.: 50.00%] [G loss: 0.912396]
4319 [D loss: 0.681999, acc.: 59.38%] [G loss: 0.782019]
4320 [D loss: 0.660250, acc.: 68.75%] [G loss: 0.849516]
4321 [D loss: 0.647918, acc.: 62.50%] [G loss: 0.835052]
4322 [D loss: 0.608616, acc.: 59.38%] [G loss: 0.895051]
4323 [D loss: 0.657808, acc.: 56.25%] [G loss: 0.910868]
4324 [D loss: 0.661293, acc.: 56.25%] [G loss: 0.869086]
4325 [D loss: 0.649339, acc.: 62.50%] [G loss: 0.887848]
4326 [D loss: 0.640021, acc.: 56.25%] [G loss: 0.873431]
4327 [D loss: 0.657017, acc.: 53.12%] [G loss: 0.967658]
4328 [D loss: 0.577405, acc.: 75.00%] [G loss: 0.957724]
4329 [D loss: 0.533515, acc.: 71.88%] [G loss: 0.899112]
4330 [D loss: 0.590686, acc.: 6

4458 [D loss: 0.604735, acc.: 71.88%] [G loss: 0.935564]
4459 [D loss: 0.503187, acc.: 78.12%] [G loss: 0.923182]
4460 [D loss: 0.654362, acc.: 56.25%] [G loss: 0.965137]
4461 [D loss: 0.606301, acc.: 68.75%] [G loss: 0.947340]
4462 [D loss: 0.654467, acc.: 59.38%] [G loss: 1.004506]
4463 [D loss: 0.792650, acc.: 34.38%] [G loss: 0.922248]
4464 [D loss: 0.650057, acc.: 62.50%] [G loss: 0.883116]
4465 [D loss: 0.658648, acc.: 65.62%] [G loss: 0.828844]
4466 [D loss: 0.666666, acc.: 62.50%] [G loss: 0.907420]
4467 [D loss: 0.624521, acc.: 59.38%] [G loss: 0.887716]
4468 [D loss: 0.613191, acc.: 65.62%] [G loss: 0.879691]
4469 [D loss: 0.658567, acc.: 62.50%] [G loss: 0.844408]
4470 [D loss: 0.623963, acc.: 68.75%] [G loss: 0.837099]
4471 [D loss: 0.563051, acc.: 71.88%] [G loss: 0.864522]
4472 [D loss: 0.678240, acc.: 53.12%] [G loss: 0.811397]
4473 [D loss: 0.652131, acc.: 59.38%] [G loss: 0.813446]
4474 [D loss: 0.623384, acc.: 59.38%] [G loss: 0.873443]
4475 [D loss: 0.635482, acc.: 7

4605 [D loss: 0.620337, acc.: 56.25%] [G loss: 0.859212]
4606 [D loss: 0.707861, acc.: 59.38%] [G loss: 0.885138]
4607 [D loss: 0.610184, acc.: 59.38%] [G loss: 0.952060]
4608 [D loss: 0.626531, acc.: 68.75%] [G loss: 0.829426]
4609 [D loss: 0.753597, acc.: 59.38%] [G loss: 0.900312]
4610 [D loss: 0.673812, acc.: 59.38%] [G loss: 0.845689]
4611 [D loss: 0.671811, acc.: 53.12%] [G loss: 0.904280]
4612 [D loss: 0.600948, acc.: 75.00%] [G loss: 0.892623]
4613 [D loss: 0.694901, acc.: 53.12%] [G loss: 0.892427]
4614 [D loss: 0.593293, acc.: 59.38%] [G loss: 0.878974]
4615 [D loss: 0.615668, acc.: 65.62%] [G loss: 0.824572]
4616 [D loss: 0.701813, acc.: 56.25%] [G loss: 0.853594]
4617 [D loss: 0.641663, acc.: 71.88%] [G loss: 0.944515]
4618 [D loss: 0.719706, acc.: 53.12%] [G loss: 0.866661]
4619 [D loss: 0.601522, acc.: 62.50%] [G loss: 0.864398]
4620 [D loss: 0.609974, acc.: 81.25%] [G loss: 0.886850]
4621 [D loss: 0.654827, acc.: 56.25%] [G loss: 0.870470]
4622 [D loss: 0.615856, acc.: 6

4749 [D loss: 0.632762, acc.: 71.88%] [G loss: 0.911813]
4750 [D loss: 0.717629, acc.: 59.38%] [G loss: 0.879614]
4751 [D loss: 0.693077, acc.: 62.50%] [G loss: 0.952689]
4752 [D loss: 0.612001, acc.: 65.62%] [G loss: 1.019495]
4753 [D loss: 0.586628, acc.: 71.88%] [G loss: 0.962240]
4754 [D loss: 0.564133, acc.: 78.12%] [G loss: 0.909998]
4755 [D loss: 0.739875, acc.: 43.75%] [G loss: 0.801179]
4756 [D loss: 0.657750, acc.: 56.25%] [G loss: 1.016076]
4757 [D loss: 0.617746, acc.: 65.62%] [G loss: 0.856327]
4758 [D loss: 0.705103, acc.: 56.25%] [G loss: 0.887172]
4759 [D loss: 0.716063, acc.: 56.25%] [G loss: 0.939604]
4760 [D loss: 0.659513, acc.: 62.50%] [G loss: 0.945092]
4761 [D loss: 0.627639, acc.: 62.50%] [G loss: 0.952062]
4762 [D loss: 0.762734, acc.: 50.00%] [G loss: 0.929864]
4763 [D loss: 0.696674, acc.: 46.88%] [G loss: 0.956516]
4764 [D loss: 0.569667, acc.: 65.62%] [G loss: 0.951949]
4765 [D loss: 0.626365, acc.: 68.75%] [G loss: 0.971189]
4766 [D loss: 0.811972, acc.: 4

4893 [D loss: 0.698006, acc.: 50.00%] [G loss: 0.880795]
4894 [D loss: 0.606640, acc.: 65.62%] [G loss: 0.895006]
4895 [D loss: 0.607381, acc.: 65.62%] [G loss: 0.902681]
4896 [D loss: 0.659663, acc.: 68.75%] [G loss: 0.929963]
4897 [D loss: 0.616084, acc.: 65.62%] [G loss: 0.962554]
4898 [D loss: 0.573954, acc.: 78.12%] [G loss: 0.932283]
4899 [D loss: 0.656718, acc.: 59.38%] [G loss: 0.939685]
4900 [D loss: 0.662863, acc.: 53.12%] [G loss: 0.925802]
4901 [D loss: 0.687723, acc.: 56.25%] [G loss: 0.979562]
4902 [D loss: 0.668717, acc.: 62.50%] [G loss: 0.957447]
4903 [D loss: 0.616887, acc.: 68.75%] [G loss: 0.882569]
4904 [D loss: 0.629106, acc.: 59.38%] [G loss: 0.826710]
4905 [D loss: 0.634278, acc.: 53.12%] [G loss: 0.879503]
4906 [D loss: 0.601311, acc.: 68.75%] [G loss: 0.822955]
4907 [D loss: 0.699381, acc.: 46.88%] [G loss: 0.793833]
4908 [D loss: 0.641444, acc.: 59.38%] [G loss: 0.929523]
4909 [D loss: 0.686238, acc.: 56.25%] [G loss: 0.882008]
4910 [D loss: 0.652353, acc.: 6

In [9]:
fig = plt.figure()
plt.plot([x[0] for x in gan.d_loss_real], color='black', linewidth=0.25)

plt.plot([x[1] for x in gan.d_losses], color='green', linewidth=0.25)
plt.plot([x[2] for x in gan.d_losses], color='red', linewidth=0.25)
plt.plot([x[0] for x in gan.g_losses], color='orange', linewidth=0.25)

plt.xlabel('batch', fontsize=18)
plt.ylabel('loss', fontsize=16)

plt.xlim(0, 2000)
plt.ylim(0, 2)

plt.show()

AttributeError: 'GAN' object has no attribute 'd_loss_real'

<Figure size 432x288 with 0 Axes>